In [4]:
from mlflow.tracking import MlflowClient

# MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [5]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/5', creation_time=1726421041867, experiment_id='5', last_update_time=1726421041867, lifecycle_stage='active', name='nyc-taxi-experiment-test/model-registry-example', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/4', creation_time=1725297092935, experiment_id='4', last_update_time=1725297092935, lifecycle_stage='active', name='nyc-taxi-experiment-test', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1724616958895, experiment_id='1', last_update_time=1724616958895, lifecycle_stage='active', name='test', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1724613800874, experiment_id='0', last_update_time=1724613800874, lifecycle_stage='active', name='Default', tags={}>]

In [36]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='5',
    # filter_string="metrics.rmse<6.8",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)


In [37]:
for run in runs:
    try:
        print(f"run id: {run.info.run_id} , rmse: {run.data.metrics['rmse']:.4f}")
    except:
        print(f"run id: {run.info.run_id} , rmse: {run.data.metrics['validation-rmse']:.4f}")

run id: a1f019666b2847d08db251316ee853d1 , rmse: 6.3850
run id: 364159005fea4c7696481ef26a455881 , rmse: 6.5862
run id: 62fdf9d14629491f84d8e0c3c9631ebc , rmse: 6.8443
run id: d764fceeb1234745b1d3da8e3a8ded77 , rmse: 991.7551
run id: 31d47de8fda24adcb8d96abb4d5be019 , rmse: 6.1657


#### example of registering model through api

In [38]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [39]:
run_id = '364159005fea4c7696481ef26a455881'
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri,name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2024/09/15 18:33:50 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 3
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1726425230253, current_stage='None', description='', last_updated_timestamp=1726425230253, name='nyc-taxi-regressor', run_id='364159005fea4c7696481ef26a455881', run_link='', source='mlflow-artifacts:/5/364159005fea4c7696481ef26a455881/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>

#### Changing stages

In [43]:
model_name = 'nyc-taxi-regressor'
latest_version = client.get_latest_versions(name=model_name)

for version in latest_version:
    print(f"version:{version.version}, stage: {version.current_stage}")
    ## Note: if there are two models in 'staging' it will only show the most recent 

version:1, stage: Production
version:3, stage: Staging


/tmp/ipykernel_29095/3424221195.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_version = client.get_latest_versions(name=model_name)


In [42]:
client.transition_model_version_stage(
    name = model_name,
    version = 3,
    stage='Staging',
    archive_existing_versions=False
    
)

/tmp/ipykernel_29095/737636949.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1726425230253, current_stage='Staging', description='', last_updated_timestamp=1726425639617, name='nyc-taxi-regressor', run_id='364159005fea4c7696481ef26a455881', run_link='', source='mlflow-artifacts:/5/364159005fea4c7696481ef26a455881/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='3'>